In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np

import io
import re
import string
import tqdm
from tensorflow.keras import layers

import seaborn as sns
from sklearn.metrics import pairwise

import os
import tensorboard
from tensorboard.plugins import projector

In [ ]:
# ds = tfds.load('multi_news', split='train', with_info=True)

# #use index to get specific document
# index = 20
# count = 0
# with tf.Graph().as_default():
#     numpy_imgs = next(iter(ds))
#     # numpy_imgs = tfds.as_numpy(ds)
# count = 0
# document = []
# summary = []
# for x in numpy_imgs:
#     count += 1
#     if count == index:
#         # tf.print(x["document"])
#         # print("\n")
#         # print("\n")
#         # print("SUMMARY")
#         # tf.print(x["summary"])

#         document = x["document"]
#         summary = x["summary"]
#         break
# document = bytes(document.numpy())
# document = [document.decode("utf-8")]

# summary = bytes(summary.numpy())
# summary = [summary.decode("utf-8")]
# #create vocab
# d_tokens = document[0].lower().split()
# s_tokens = summary[0].lower().split()
# tokens = d_tokens + s_tokens
# vocab, index = {}, 1
# vocab["<pad>"] = 0
# for token in tokens:
#     if token not in vocab:
#         vocab[token] = index
#         index = index + 1

# inverse_vocab = {index: token for token, index in vocab.items()}
# example = [vocab[word] for word in s_tokens]
# embed = hub.load("https://tfhub.dev/google/nnlm-en-dim128/2")
# embeddings = embed(["cat is on the mat", "dog is in the fog"])
# print(embeddings)
# # stuff to do
# # get vocabulary
# # display output in a visual way
# #lean word2vec

# preprocess = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
# bert = hub.load('https://tfhub.dev/google/experts/bert/wiki_books/2')

# sentences = [
#   "Here We Go Then, You And I is a 1999 album by Norwegian pop artist Morten Abel. It was Abel's second CD as a solo artist.",
#   "The album went straight to number one on the Norwegian album chart, and sold to double platinum.",
#   "Among the singles released from the album were the songs \"Be My Lover\" and \"Hard To Stay Awake\".",
#   "Riccardo Zegna is an Italian jazz musician.",
#   "Rajko Maksimović is a composer, writer, and music pedagogue.",
#   "One of the most significant Serbian composers of our time, Maksimović has been and remains active in creating works for different ensembles.",
#   "Ceylon spinach is a common name for several plants and may refer to: Basella alba Talinum fruticosum",
#   "A solar eclipse occurs when the Moon passes between Earth and the Sun, thereby totally or partly obscuring the image of the Sun for a viewer on Earth.",
#   "A partial solar eclipse occurs in the polar regions of the Earth when the center of the Moon's shadow misses the Earth.",
# ]

# wordArray = []

# for i in sentences:
#   words = i.split()
#   for w in words:
#     w = w.replace(",", "")
#     w = w.replace(".", "")
#     wordArray.append(w)

# print("word array")
# print(wordArray)

# bert_inputs = preprocess(sentences)
# bert_outputs = bert(bert_inputs)
# pooled_output = bert_outputs['pooled_output']
# sequence_output = bert_outputs['sequence_output']

# print('\nPooled output:')
# print(pooled_output)
# print('\nSequence output:')
# print(sequence_output)

# def plot_similarity(features, labels):
#   """Plot a similarity matrix of the embeddings."""
#   cos_sim = pairwise.cosine_similarity(features)
#   sns.set(font_scale=1.2)
#   cbar_kws=dict(use_gridspec=False, location="left")
#   g = sns.heatmap(
#       cos_sim, xticklabels=labels, yticklabels=labels,
#       vmin=0, vmax=1, cmap="Blues", cbar_kws=cbar_kws)
#   g.tick_params(labelright=True, labelleft=False)
#   g.set_yticklabels(labels, rotation=0)
#   g.set_title("Semantic Textual Similarity")

# plot_similarity(bert_outputs["pooled_output"], wordArray)

In [57]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE
num_ns = 4

# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for vocab_size tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=SEED,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')

class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1,
                                       name="w2v_context")

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

In [58]:
def createWordVectors(textIdx, useBoard):
    file_path = textIdx
    text_ds = tf.data.TextLineDataset(file_path).filter(lambda x: tf.cast(tf.strings.length(x), bool))
    
    # Define the vocabulary size and number of words in a sequence.
    vocab_size = 4096
    sequence_length = 300

    # Use the TextVectorization layer to normalize, split, and map strings to
    # integers. Set output_sequence_length length to pad all samples to same length.
    vectorize_layer = layers.TextVectorization(
        standardize=custom_standardization,
        max_tokens=vocab_size,
        output_mode='int',
        output_sequence_length=sequence_length)

    vectorize_layer.adapt(text_ds.batch(1024))

    inverse_vocab = vectorize_layer.get_vocabulary()

    text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

    sequences = list(text_vector_ds.as_numpy_iterator())
    
    targets, contexts, labels = generate_training_data(
        sequences=sequences,
        window_size=2,
        num_ns=4,
        vocab_size=vocab_size,
        seed=SEED)

    contexts = np.array(contexts)[:,:,0]
    labels = np.array(labels)

    # BATCH_SIZE = 1024
    # BUFFER_SIZE = 10000
    BATCH_SIZE = 10
    BUFFER_SIZE = 15
    dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

    dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)

    embedding_dim = 128
    word2vec = Word2Vec(vocab_size, embedding_dim)
    word2vec.compile(optimizer='adam',
                    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])
    
    if useBoard:
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
        word2vec.fit(dataset, epochs=1, callbacks=[tensorboard_callback])
    else:
        word2vec.fit(dataset, epochs=1)

    tWeights = word2vec.get_layer('w2v_embedding').get_weights()[0]
    cWeights = word2vec.get_layer('w2v_context').get_weights()[0]
    vocab = vectorize_layer.get_vocabulary()

    # tf_weight = tf.Variable(tWeights)
    # checkpoint = tf.train.Checkpoint(embedding=tf_weight)
    # checkpoint.save(os.path.join('logs', "embedding.ckpt"))
    return tWeights, cWeights, vocab, sequences

In [59]:
# doc_weights, doc_vocab = createWordVectors("document.txt", False)
tWeights, cWeights, vocab, sequences = createWordVectors("summary.txt", False)


100%|██████████| 1/1 [00:00<00:00, 143.26it/s]


5/5 [==============================] - 1s 11ms/step - loss: 1.6096 - accuracy: 0.2200


In [ ]:
file_path = 'document.txt'
text_ds = tf.data.TextLineDataset(file_path).filter(lambda x: tf.cast(tf.strings.length(x), bool))
text_ds = text_ds.enumerate()
docTxt = []
for i in text_ds.as_numpy_iterator():
    line = i[1].decode().split()
    for l in line:
        docTxt.append(l)

In [85]:
import collections

#target and context weights for the summary model
print(tWeights.shape)
print(cWeights.shape)
vocab = np.array(vocab)
sequences = np.array(sequences)

#find the key words in the document
docFeq = collections.Counter(docTxt)
print(docFeq)    




(4096, 128)
(4096, 128)
Counter({'the': 182, 'a': 106, 'of': 101, 'and': 93, 'in': 73, 'he': 69, 'to': 60, 'that': 51, 'is': 46, 'his': 38, 'it': 34, 'was': 32, 'as': 30, 'on': 29, 'for': 28, 'i': 27, 'eco': 21, 'but': 21, 'with': 19, 'not': 19, 'are': 18, 'at': 17, 'name': 17, 'an': 17, 'have': 17, 'has': 16, 'be': 15, 'who': 14, 'by': 13, 'they': 13, 'book': 12, 'had': 12, 'him': 11, 'from': 11, '–': 11, 'my': 11, 'intellectual': 10, 'you': 10, 'so': 9, 'when': 9, 'wrote': 9, 'this': 9, 'been': 8, 'or': 8, 'berlusconi': 8, 'italian': 7, 'novel': 7, 'were': 7, 'too': 7, 'there': 7, 'always': 7, '"i': 7, 'them': 7, 'people': 7, 'new': 7, 'rose': 7, 'conspiracy': 7, 'novels': 7, 'like': 6, '"the': 6, 'also': 6, 'books,': 6, 'books': 6, 'up': 6, 'most': 6, "don't": 6, 'some': 6, 'says': 6, "eco's": 6, 'after': 5, 'made': 5, 'novel,': 5, 'already': 5, 'how': 5, 'italy': 5, 'take': 5, 'will': 5, 'years': 5, 'read': 5, 'even': 5, 'do': 5, 'says.': 5, 'file': 4, 'writer': 4, 'umberto': 4, 'k

In [40]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(sum_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = sum_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [48]:
# Save the weights we want to analyze as a variable. Note that the first
# value represents any unknown word, which is not in the metadata, here
# we will remove this value.
weights = tf.Variable(sum_weights)
# Create a checkpoint from embedding, the filename and key are the
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join("logs", "embedding.ckpt"))

# Set up config.
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings('logs', config)

In [42]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins import projector

(train_data, test_data), info = tfds.load(
    "imdb_reviews/subwords8k",
    split=(tfds.Split.TRAIN, tfds.Split.TEST),
    with_info=True,
    as_supervised=True,
)
encoder = info.features["text"].encoder

# Shuffle and pad the data.
train_batches = train_data.shuffle(1000).padded_batch(
    10, padded_shapes=((None,), ())
)
test_batches = test_data.shuffle(1000).padded_batch(
    10, padded_shapes=((None,), ())
)
train_batch, train_labels = next(iter(train_batches))

# Create an embedding layer.
embedding_dim = 16
embedding = tf.keras.layers.Embedding(encoder.vocab_size, embedding_dim)
# Configure the embedding layer as part of a keras model.
model = tf.keras.Sequential(
    [
        embedding, # The embedding layer should be the first layer in a model.
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1),
    ]
)

# Compile model.
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

# Train model for one epoch.
history = model.fit(
    train_batches, epochs=1, validation_data=test_batches, validation_steps=20
)

# Set up a logs directory, so Tensorboard knows where to look for files.
log_dir='logs'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Save Labels separately on a line-by-line manner.
with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
  for subwords in encoder.subwords:
    f.write("{}\n".format(subwords))
  # Fill in the rest of the labels with "unknown".
  for unknown in range(1, encoder.vocab_size - len(encoder.subwords)):
    f.write("unknown #{}\n".format(unknown))


# Save the weights we want to analyze as a variable. Note that the first
# value represents any unknown word, which is not in the metadata, here
# we will remove this value.
weights = tf.Variable(model.layers[0].get_weights()[0][1:])
# Create a checkpoint from embedding, the filename and key are the
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

# Set up config.
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

Dl Completed...: 100%|██████████| 1/1 [00:14<00:00, 14.67s/ url]


Dataset imdb_reviews downloaded and prepared to C:\Users\Brad\tensorflow_datasets\imdb_reviews\subwords8k\1.0.0. Subsequent calls will reuse this data.
2500/2500 [==============================] - 7s 3ms/step - loss: 0.4999 - accuracy: 0.7047 - val_loss: 0.3421 - val_accuracy: 0.8600


UnicodeEncodeError: 'charmap' codec can't encode character '\x96' in position 1: character maps to <undefined>